In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime
from time import time
from datetime import timedelta
import mercury as mr 
import warnings
import altair as alt
warnings.filterwarnings('ignore')

In [2]:
app = mr.App(title="Coil Spring rate Comparisons")

mercury.App

<font size="7">Setup analyzer: Uncover your ideal configuration</font>

<font size="2">To use: enter your details on the left and change the spring rate until you find an appropriate spring for your mountain bike, travel and weight</font>

<font size="2">Credit: Rowland Jowett for the original very useful prototype.


In [23]:
#read the results data
#Stroke = rear shock stroke (mm). Shock length at full extension - shock length at full compression. Typically ~ 76mm
#Travel = rear wheel vertical travel, mm. Typically 160mm. 
#Weight = rider weight, kg. 
#Discipline = {DH, Enduro, Other}
#Speed = 1-10. This is relative to Mens world cup DH speed (10). 1 = beginner. 6 = mid pack male grassroots DH race.
#Spring_rate = spring rate in lbs/in
#name = rider name
df = pd.read_csv("Data.csv", index_col=1)
df = df.reset_index()

user_discipline = mr.Select(value="Enduro", choices=['Enduro', 'DH'], label="Discipline")
user_stroke = mr.Slider(value=63, min=20, max=100, label="Rear shock stroke (mm)", step=0.5)
user_travel = mr.Slider(value=160, min=100, max=250, label="Rear wheel vertical travel (mm)", step=5)
user_weight = mr.Slider(value=80, min=20, max=200, label="Rider weight (Kg)", step=1)
user_spring_rate = mr.Slider(value=434, min=200, max=800, label="Spring_rate lbs/in", step=5)
user_speed_rating = mr.Slider(value=5, min=1, max=10, label="Rider speed, Mens WCDH = 10", step=1)
user_name = mr.Text(value='Jane Doe', label='Name')
#button_upload = mr.Button(label="Upload to database", style="primary")


def motion_ratio(travel, stroke):
    return travel/stroke

def spring_rate_at_wheel(spring, motion_ratio):
    return spring/(motion_ratio**2)

def spring_rate_at_wheel_normlaised_75kg(spring_rate_at_wheel, weight):
    return spring_rate_at_wheel*75/weight

def energy_at_max_travel(spring_rate, travel):
    return 0.5*(spring_rate*175.126835)*((travel/1000)**2)

def huck_height(energy_at_max_travel, weight):
    return energy_at_max_travel/(0.6*weight*9.81)

def add_label(name, spring):
    return name + " " + str(spring) + "lbs/in"
    
def add_calucated_quantitles(df):
    df['Motion_ratio'] = motion_ratio(df['Travel'], df['Stroke'])
    df['Spring_rate_at_wheel'] = spring_rate_at_wheel(df['Spring_rate'],df['Motion_ratio'])
    df['Spring_rate_at_wheel_normalised_75kg'] = spring_rate_at_wheel_normlaised_75kg(df['Spring_rate_at_wheel'],df['Weight'])
    df['Energy_at_max_travel'] = energy_at_max_travel(df['Spring_rate'], df['Travel'])
    df['Huck_height_(m)'] = huck_height(df['Energy_at_max_travel'], df['Weight'])
    df['LabelX'] = np.vectorize(add_label)(df['Name'], df['Spring_rate'])
    

data = {
        'Discipline': ['Entered data'],
        'Name': [user_name.value],
        'Stroke': [user_stroke.value],
        'Travel': [user_travel.value],
        'Spring_rate': [user_spring_rate.value],
        'Weight': [user_weight.value],
        'Speed_rating': [user_speed_rating.value],
    }   

df_user = pd.DataFrame(data)
add_calucated_quantitles(df)
add_calucated_quantitles(df_user)
df_user['plot_point_size'] = 3
df['plot_point_size'] = 2
df_combined = pd.concat([df, df_user])


# Make the chart
points = alt.Chart(df, title='Normalised Rider Weight Spring Rate vs Bike Max Travel').mark_circle().encode(
    alt.X('Travel:Q',axis=alt.Axis(title ='Travel_')).scale(zero=False),
    alt.Y('Spring_rate_at_wheel_normalised_75kg:Q', axis=alt.Axis(title ='Spring_rate_at_wheel_normalised_75kg_')).scale(zero=False),
    color='Discipline:N',
    tooltip=['Name', 'Weight', 'Spring_rate', 'Speed_rating', 'Discipline'],
)
points_user = alt.Chart(df_user, title='Normalised Rider Weight Spring Rate vs Bike Max Travel').mark_circle().encode(
    alt.X('Travel:Q',axis=alt.Axis(title ='Travel_')).scale(zero=False),
    alt.Y('Spring_rate_at_wheel_normalised_75kg:Q', axis=alt.Axis(title ='Spring_rate_at_wheel_normalised_75kg_')).scale(zero=False),
    color='Discipline:N',
    tooltip=['Name', 'Weight', 'Spring_rate', 'Speed_rating', 'Discipline'],
    size=alt.value(200)
)

labels =  alt.Chart(df_combined).mark_text(align='left', baseline='middle', dx=4).encode(alt.X('Travel:Q').scale(zero=False),
    alt.Y('Spring_rate_at_wheel_normalised_75kg:Q').scale(zero=False),
    text='LabelX:N')

reg = alt.Chart(df).mark_circle().encode(
    alt.X('Travel:Q',axis=alt.Axis(title ='Travel_')).scale(zero=False),
    alt.Y('Spring_rate_at_wheel_normalised_75kg:Q', axis=alt.Axis(title ='Spring_rate_at_wheel_normalised_75kg_')).scale(zero=False),
    color='Discipline:N',
    tooltip=['Name', 'Weight', 'Spring_rate', 'Speed_rating', 'Discipline']
).transform_regression('Travel', 'Spring_rate_at_wheel_normalised_75kg').mark_line(
     opacity=0.50, 
     shape='mark'
).transform_fold(
     ["best fit line"], 
     as_=["Regression", "y"]
).encode(alt.Color("Regression:N"))


charts = (points + points_user + reg + labels).properties(width="container").interactive()


# Make the chart
huck_height_chart = alt.Chart(df, title='Huck_height_(m) ').mark_circle().encode(
    alt.X('Travel:Q').scale(zero=False),
    alt.Y('Huck_height_(m):Q').scale(zero=False),
    color='Discipline:N',
    tooltip=['Name', 'Weight', 'Spring_rate', 'Speed_rating', 'Discipline']
).properties(
    width="container"
)

huck_height_chart_user = alt.Chart(df_user, title='Huck_height_(m) ').mark_circle().encode(
    alt.X('Travel:Q').scale(zero=False),
    alt.Y('Huck_height_(m):Q').scale(zero=False),
    color='Discipline:N',
    tooltip=['Name', 'Weight', 'Spring_rate', 'Speed_rating', 'Discipline'],
    size=alt.value(200)
).properties(
    width="container"
)

labels_h =  alt.Chart(df_combined).mark_text(align='left', baseline='middle', dx=4).encode(alt.X('Travel:Q').scale(zero=False),
    alt.Y('Huck_height_(m):Q').scale(zero=False),
    text='LabelX:N')

reg_h = alt.Chart(df).mark_circle().encode(
    alt.X('Travel:Q',axis=alt.Axis(title ='Travel_')).scale(zero=False),
    alt.Y('Huck_height_(m):Q', axis=alt.Axis(title ='Huck_height_(m)')).scale(zero=False),
    color='Discipline:N',
    tooltip=['Name', 'Weight', 'Spring_rate', 'Speed_rating', 'Discipline']
).transform_regression('Travel', 'Huck_height_(m)').mark_line(
     opacity=0.50, 
     shape='mark'
).transform_fold(
     ["best fit line"], 
     as_=["Regression", "y"]
).encode(alt.Color("Regression:N"))

charts2 = (huck_height_chart + huck_height_chart_user + reg_h + labels_h).interactive()
charts

mercury.Select

mercury.Slider

mercury.Slider

mercury.Slider

mercury.Slider

mercury.Slider

mercury.Text

alt.LayerChart(...)

In [24]:
charts2

alt.LayerChart(...)

<font size="4">
Definitions: </font>


motion_ratio(travel, stroke):
    return travel/stroke

spring_rate_at_wheel(spring, motion_ratio):
    return spring/(motion_ratio**2)

spring_rate_at_wheel_normlaised_75kg(spring_rate_at_wheel, weight):
    return spring_rate_at_wheel*75/weight

energy_at_max_travel(spring_rate, travel):
    return 0.5*(spring_rate*175.126835)*((travel/1000)**2)

Huck_height is the height you could drop rider and bike from and all energy be contained in spring without bottoming out (assumes 60% of weight on rear wheel):
huck_height(energy_at_max_travel, weight):
    return energy_at_max_travel/(0.6*weight*9.81)

Any questions, please email mulholland.william@gmail.com with the subject: Coil spring rate comparisons.

If this page is of more than fleeting interest, please consider donating to the air ambulance service: https://theairambulanceservice.org.uk

 </font>